# Hindcast Summary Demo

This example shows how to access Salient's rigorous data validation system and highlight areas of particular interest.


In [ ]:
import os
import sys

import pandas as pd

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

sk.set_file_destination("hindcast_summary_example", force=False)
sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD", verbose=False)

## Validate

The `hindcast_summary` function is a convenience interface to the `hindast_summary` [API endpoint](https://api.salientpredictions.com/v2/documentation/api/#/Validation/hindcast_summary) which delivers accuracy statistics. This example vectorizes multiple requests, downloads them in parallel, and concatenates all the results into a single file.


In [ ]:
score_file = sk.hindcast_summary(
    loc=sk.Location(region=["united states", "germany"]),
    variable=["temp", "precip"],  # Vectorize multi-variable
    metric="crps_skill_score",
    reference="clim",
    season=["DJF", "JJA"],  # Vectorize meteorological winter and summer
    version="-default",  # this is vectorizable
    force=False,
    verbose=False,
)

print(score_file)
print(pd.read_csv(score_file))

hindcast_summary_example/hindcast_summary_dda9e7bca8ffdd70d3268eb30d8683d2.csv
            Lead      Reference Model  Reference CRPS  Salient CRPS  Salient CRPS Skill Score (%)         region season variable
0         Week 1  30 year Climatology           2.568         0.786                         68.81  united states    DJF     temp
1         Week 2  30 year Climatology           2.567         1.786                         30.67  united states    DJF     temp
2         Week 3  30 year Climatology           2.567         2.313                         10.04  united states    DJF     temp
3         Week 4  30 year Climatology           2.567         2.462                          4.21  united states    DJF     temp
4         Week 5  30 year Climatology           2.567         2.476                          3.74  united states    DJF     temp
..           ...                  ...             ...           ...                           ...            ...    ...      ...
91       Month 3  

# Simplify Score File

The native score file returned by `hindcast_summary` is "data-long", which means that each `Lead` time gets a row of its own. To make the data more compact, we can focus only on the 5th "skill score" column and transpose all of its rows to columns. Use the sdk function `transpose_hindcast_summary()` to transform the hindcast csv.

The min_score argument converts any score below the threshold value into `NaN`. By default, this is set to zero. In this example, setting it to `5.0` creates a "bright spot" analysis which highlights the examples where the Salient forecast significantly outperforms the reference model.


In [ ]:
scores = sk.transpose_hindcast_summary(score_file, min_score=5.0)

save_file = os.path.join(sk.get_file_destination(), "hindcast_summary_transposed.csv")
scores.to_csv(save_file)

print(save_file)
print(scores)

hindcast_summary_example/hindcast_summary_transposed.csv
Lead                           Week 1  Week 2  Week 3  Week 4  Week 5  Month 1  Month 2  Month 3  Months 1-3  Months 4-6  Months 7-9  Months 10-12   mean
region        season variable                                                                                                                            
germany       DJF    precip     52.66   12.66     NaN     NaN     NaN    14.06      NaN      NaN         NaN         NaN         NaN           NaN    NaN
                     temp       73.36   33.48   10.56    6.80    6.27    33.21     7.91     8.50       12.23       10.67        6.62          5.87   7.92
              JJA    precip     37.68    6.95     NaN     NaN     NaN    11.93      NaN      NaN         NaN         NaN         NaN           NaN    NaN
                     temp       72.63   30.09   12.40    9.33    9.35    35.55    19.87    19.14       20.70       22.30       21.96         22.30  14.15
united states DJF  